### Include registration modules

In [1]:
import copy

from functools import partial
from time import time

from collections.abc import Callable

import numpy as np

import open3d
import open3d.geometry as geom
import open3d.pipelines.registration as reg
import open3d.visualization as vis
import open3d.utility as util

from mynd.registration import (
    MultiTargetIndex,
    generate_cascade_indices,
)

from mynd.registration import (
    downsample_point_cloud,
    estimate_point_cloud_normals,
)

from mynd.registration import (
    register_icp,
    register_colored_icp,
    build_pose_graph,
    optimize_pose_graph,
)

from mynd.spatial import decompose_transformation

from mynd.visualization import (
    visualize_registration,
    create_subplots,
    trace_registration_result,
)

from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


### Load environment and configure data loaders

In [2]:
from pathlib import Path

from mynd.geometry import PointCloud, PointCloudLoader
from mynd.io import read_point_cloud, create_point_cloud_loader

DATA_DIR: Path = Path("/home/martin/dev/mynd/.cache")

point_cloud_files: dict = {
    0: DATA_DIR / Path("qdc5ghs3_20100430_024508.ply"),
    1: DATA_DIR / Path("qdc5ghs3_20120501_033336.ply"),
    2: DATA_DIR / Path("qdc5ghs3_20130405_103429.ply"),
    3: DATA_DIR / Path("qdc5ghs3_20210315_230947.ply"),
}

loaders: dict[int, PointCloudLoader] = {
    key: create_point_cloud_loader(path)
    for key, path in point_cloud_files.items()
    if path.exists()
}

count = len(loaders)
if count < 2:
    logger.error(f"invalid number of point clouds for registration: {count}")

### Build registration modules from config

In [3]:
from dataclasses import dataclass
from pathlib import Path
from typing import TypeAlias

from mynd.io import read_config
from mynd.utils.log import logger

# Processors and registrators
from mynd.registration import (
    RegistrationResult,
    PointCloudProcessor,
    GlobalRegistrator,
    IncrementalRegistrator,
)

# Builders
from mynd.registration import (
    build_point_cloud_processor,
    build_icp_registrator,
    build_ransac_registrator,
)

from mynd.registration import log_registration_result

from mynd.registration import RegistrationPipeline, apply_registration_pipeline


# TODO: Create ransac registrator
preprocessor: PointCloudProcessor = build_point_cloud_processor(
    {
        "downsample": {"spacing": 0.20},
        "estimate_normals": {"radius": 0.40, "neighbours": 30},
    }
)

ransac_registrator: GlobalRegistrator = build_ransac_registrator(
    {
        "feature": {"radius": 2.00, "neighbours": 200},
        "estimator": {"with_scaling": True},
        "validators": {
            "distance_threshold": 0.15,
            "edge_threshold": 0.95,
            "normal_threshold": 5.0,
        },
        "convergence": {"max_iteration": 1000, "confidence": 1.0},
        "algorithm": {
            "distance_threshold": 0.15,
            "sample_count": 3,
            "mutual_filter": True,
        },
    }
)


logger.info(preprocessor)
logger.info(ransac_registrator)

# pipeline: RegistrationPipeline = RegistrationPipeline(initializer)
# result: RegistrationResult = apply_registration_pipeline(pipeline, source=source, target=target)

target_pre: PointCloud = preprocessor(loaders.get(0)().unwrap())
source_pre: PointCloud = preprocessor(loaders.get(1)().unwrap())

2024-11-03 22:31:01.375 | INFO     | __main__:<module>:43 - <function build_point_cloud_processor.<locals>.preprocess_point_cloud at 0x7d49c70e6b60>
2024-11-03 22:31:01.375 | INFO     | __main__:<module>:44 - <function create_ransac_registrator.<locals>.ransac_registrator_wrapper at 0x7d49c70e6c00>


In [4]:
with open3d.utility.VerbosityContextManager(
    open3d.utility.VerbosityLevel.Debug
) as cm:

    result: RegistrationResult = ransac_registrator(
        target=target_pre, source=source_pre
    )

    logger.info(result)

2024-11-03 22:31:16.799 | INFO     | mynd.registration.feature_registrators:register_features_ransac:214 - PointCloud with 34412 points.
2024-11-03 22:31:16.800 | INFO     | mynd.registration.feature_registrators:register_features_ransac:215 - PointCloud with 29698 points.
2024-11-03 22:31:17.211 | INFO     | mynd.registration.feature_registrators:register_features_ransac:223 - Feature class with dimension = 33 and num = 34412
Access its data via data member.
2024-11-03 22:31:17.211 | INFO     | mynd.registration.feature_registrators:register_features_ransac:224 - Feature class with dimension = 33 and num = 29698
Access its data via data member.
2024-11-03 22:31:17.212 | INFO     | mynd.registration.feature_registrators:match_features_ransac:163 - PointCloud with 34412 points.
2024-11-03 22:31:17.212 | INFO     | mynd.registration.feature_registrators:match_features_ransac:164 - PointCloud with 29698 points.
2024-11-03 22:31:17.832 | INFO     | mynd.registration.feature_registrators:ma

[Open3D DEBUG] 7783 correspondences remain after mutual filter
[Open3D DEBUG] RANSAC exits after 0 validations. Best inlier ratio 0.000000e+00, RMSE 0.000000e+00
